In [2]:
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"

# Agents

The primary idea is to leverage Large Language Models (LLMs) for the purpose of reasoning and planning sequences of actions in a logical order, utilizing suitable tools. Upon receiving a query, the LLM utilizes its language comprehension abilities to grasp the query, determine the course of action, and execute it using the available tools.

In [3]:
import os
import sys

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

/Users/L024258/lilly_work/github-copilot/exploration/notebooks/..


In [4]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')

In [5]:
os.environ["AZURE_OPENAI_API_KEY"] = config('OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')
os.environ["COHERE_API_KEY"] = config('COHERE_TOKEN')

## Define Tools

### Langchain Pre-defined Tools

#### Arxiv Search Tool

In [6]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun
arxiv_search = ArxivQueryRun()

In [7]:
arxiv_search('LLM Agents')

"Published: 2024-03-18\nTitle: EnvGen: Generating and Adapting Environments via LLMs for Training Embodied Agents\nAuthors: Abhay Zala, Jaemin Cho, Han Lin, Jaehong Yoon, Mohit Bansal\nSummary: Recent SOTA approaches for embodied learning via interaction directly employ\nlarge language models (LLMs) as agents to determine the next steps in an\nenvironment. Due to their world knowledge and reasoning capabilities, LLM\nagents achieve stronger performance than previous smaller agents based on\nreinforcement learning (RL); however, frequently calling LLMs is slow and\nexpensive. Instead of directly employing LLMs as agents, can we use LLMs'\nreasoning capabilities to adaptively create training environments to help\nsmaller embodied RL agents learn useful skills that they are weak at? We\npropose EnvGen, a novel framework to address this question. First, we prompt an\nLLM to generate training environments that allow agents to quickly learn\ndifferent tasks in parallel. Concretely, the LLM i

### Custom Tools

#### Highlights Generator

In [39]:
from langchain.agents import tool
from src.text_summarizer import TextSummarizer, TextSummarizerAPIInputs

@tool
def get_highlights(input_text: str) -> int:
    """Returns the summary highlights of the text."""
    api_inputs = {
        "input_string": input_text,
        "input_type": "text",
        "summary_type": "highlights",
        "llm_model": "gpt-4",
        "user_message": "Generate key highlights from the provided text:",
    }

    api_obj = TextSummarizerAPIInputs(**api_inputs)
    obj = TextSummarizer(api_obj)
    return obj.summarize()


#### Short Summary Generator

In [40]:
from langchain.agents import tool
from src.text_summarizer import TextSummarizer, TextSummarizerAPIInputs

@tool
def get_short_summary(input_text: str) -> int:
    """Returns the short summary of the text."""
    api_inputs = {
        "input_string": input_text,
        "input_type": "text",
        "summary_type": "short",
        "llm_model": "gpt-4",
        "user_message": "Generate concise summary of the provided text:",
    }

    api_obj = TextSummarizerAPIInputs(**api_inputs)
    obj = TextSummarizer(api_obj)
    return obj.summarize()

In [41]:
tools = [arxiv_search, pubmed_search, get_highlights, get_short_summary]

### Creating the Agent

#### Initialising the LLM

In [42]:
from connectors.llms.models import ModelSelector
llm = ModelSelector(auth_config).get_model('gpt-4')

#### Setting up the prompt

In [45]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are very powerful research assistant.
            Your job is to assist in literature review on a given topic.
            You can search for research papers, on ArXiv or Pubmed.
            Search, Summarize, or generate key highlights of the the research papers as requested.""",
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

`create_openai_functions_agent`:  A Runnable sequence representing an agent. It takes as input all the same input variables as the prompt passed in does. It returns as output either an Agent's action or finish.

In [46]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

`AgentExecutor`: This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats.

In [47]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [48]:
result = agent_executor.invoke({"input": "hi!"})
print(result['output'])



> Entering new AgentExecutor chain...
Hello! How can I help you today? If you need assistance with a literature review or research on a specific topic, please let me know.

> Finished chain.
Hello! How can I help you today? If you need assistance with a literature review or research on a specific topic, please let me know.


In [ ]:
result

In [51]:
print(agent_executor.invoke({"input": "Short summary on drug discovery using AI."})['output'])



> Entering new AgentExecutor chain...

Invoking: `pub_med` with `drug discovery using AI`


Published: 2024-03-06
Title: Discovery of 3-hydroxymethyl-azetidine derivatives as potent polymerase theta inhibitors.
Copyright Information: Copyright © 2024 Elsevier Ltd. All rights reserved.
Summary::
Inhibition of the low fidelity DNA polymerase Theta (Polθ) is emerging as an attractive, synthetic-lethal antitumor strategy in BRCA-deficient tumors. Here we report the AI-enabled development of 3-hydroxymethyl-azetidine derivatives as a novel class of Polθ inhibitors featuring central scaffolding rings. Structure-based drug design first identified A7 as a lead compound, which was further optimized to the more potent derivative B3 and the metabolically stable deuterated compound C1. C1 exhibited significant antiproliferative properties in DNA repair-compromised cells and demonstrated favorable pharmacokinetics, showcasing that 3-hydroxymethyl-azetidine is an effective bio-isostere of pyrrolid

### Adding memory to the Agent

In [52]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [53]:
message_history = ChatMessageHistory()

In [54]:
new_agent = agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [55]:
result = new_agent.invoke({"input": "Research article on drug discovery using AI?"},
                          config={"configurable": {"session_id": "1234"}},)
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'drug discovery using AI'}`


Published: 2023-07-13
Title: Artificial Intelligence for Drug Discovery: Are We There Yet?
Authors: Catrin Hasselgren, Tudor I. Oprea
Summary: Drug discovery is adapting to novel technologies such as data science,
informatics, and artificial intelligence (AI) to accelerate effective treatment
development while reducing costs and animal experiments. AI is transforming
drug discovery, as indicated by increasing interest from investors, industrial
and academic scientists, and legislators. Successful drug discovery requires
optimizing properties related to pharmacodynamics, pharmacokinetics, and
clinical outcomes. This review discusses the use of AI in the three pillars of
drug discovery: diseases, targets, and therapeutic modalities, with a focus on
small molecule drugs. AI technologies, such as generative chemistry, machine
learning, and multi-property optimization, have enabled sever

In [56]:
print(result)

{'input': 'Research article on drug discovery using AI?', 'chat_history': [], 'output': 'I found three research articles on drug discovery using AI:\n\n1. [Artificial Intelligence for Drug Discovery: Are We There Yet?](https://arxiv.org/abs/2107.06272) (Published: 2023-07-13)\nAuthors: Catrin Hasselgren, Tudor I. Oprea\n\nKey highlights:\n- AI is transforming drug discovery through data science, informatics, and artificial intelligence.\n- AI technologies such as generative chemistry, machine learning, and multi-property optimization have enabled several compounds to enter clinical trials.\n- The full potential of AI in drug discovery can only be realized with sufficient ground truth and appropriate human intervention at later pipeline stages.\n\n2. [Artificial Intelligence in Drug Discovery: Applications and Techniques](https://arxiv.org/abs/2111.01058) (Published: 2021-11-02)\nAuthors: Jianyuan Deng, Zhibo Yang, Iwao Ojima, Dimitris Samaras, Fusheng Wang\n\nKey highlights:\n- Various

In [57]:
result = new_agent.invoke({"input": "Generate highlights of the research article from previous search."},
                          config={"configurable": {"session_id": "1234"}},)
print(result)



> Entering new AgentExecutor chain...

Invoking: `get_highlights` with `{'input_text': '1. Artificial Intelligence for Drug Discovery: Are We There Yet? (Published: 2023-07-13)\nAuthors: Catrin Hasselgren, Tudor I. Oprea\n\nKey highlights:\n- AI is transforming drug discovery through data science, informatics, and artificial intelligence.\n- AI technologies such as generative chemistry, machine learning, and multi-property optimization have enabled several compounds to enter clinical trials.\n- The full potential of AI in drug discovery can only be realized with sufficient ground truth and appropriate human intervention at later pipeline stages.\n\n2. Artificial Intelligence in Drug Discovery: Applications and Techniques (Published: 2021-11-02)\nAuthors: Jianyuan Deng, Zhibo Yang, Iwao Ojima, Dimitris Samaras, Fusheng Wang\n\nKey highlights:\n- Various AI techniques have been used in a wide range of applications, such as virtual screening and drug design.\n- The authors discuss commo

In [58]:
print(result['output'])

Here are the highlights of the research articles:

- AI is revolutionizing drug discovery through generative chemistry, machine learning, and multi-property optimization.
- Virtual screening and drug design benefit from various AI techniques.
- AI's effectiveness relies on sufficient ground truth and human intervention.
- A new task proposes translating between drug molecules and indications.
- Large Language Models show potential in this task, but further improvements are necessary.
